In [ ]:
import pandas as pd
import numpy as np
import os 
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
pio.renderers.default = "iframe"

import pandas as pd
import numpy as np
import os
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 
import os



import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import psycopg2
import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np

import time 
import pygsheets
import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream

from scipy import stats


def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)

def create_wick_trigger(df,col = 'wick',lookback_threshold = 3600):
    
    print('CREATING WICK TRIGGER...')
    df['wick'] = 0
    df.loc[df[df['delta_vector'] < 0].index,'wick'] = df.loc[df[df['delta_vector'] < 0].index,'c'] - df.loc[df[df['delta_vector'] < 0].index,'l']
    df.loc[df[df['delta_vector'] > 0].index,'wick'] = df.loc[df[df['delta_vector'] > 0].index,'h'] - df.loc[df[df['delta_vector'] > 0].index,'c']
    df['wick_trigger'] = 0


    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)
def consecutive_candles(df,num = 10,col = 'delta_vector'):
    print('CREATING CONSECUTIVE CANDLES TRIGGER...')
    df[col + '_consecutive_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    last = -1
    counter = 0
    for i in range(arr.shape[0]):
        
        if i % 200000 == 0:
            print(i)
            
        if arr[i-1,col_loc] < 0 and arr[i,col_loc] < 0:
            counter += 1
        elif arr[i-1,col_loc] > 0 and arr[i,col_loc] > 0:
            counter += 1  
        else:
            counter = 0
            
        if counter == num:
            arr[i,-1] = 1
            counter = 0
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE OF TRIGGER DF:',df[df[col + '_consecutive_trigger'] == 1].shape)
    return df

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - looback) : i,ind_col], arr[(i - looback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def get_pearsons_corr2(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_lookup'] = 0
    df['pearsons_corr2'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('pearsons_lookup')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if i % 2 == 0:
            arr[i,new_col] = arr[i,o_col]
        else:
            arr[i,new_col] = arr[i,c_col]        
            
            
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,new_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)
def get_best_fit(df,lookback = 10):

    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,open_list,close_list,total_x):

        for i in range(total_x):
            if lst[i] <= max(open_list[i],close_list[i]) and lst[i] >= min(open_list[i],close_list[i]):
                pass
            else:
                return 0
        return 1
    print('GETTING BEST FIT INDICATOR')
    df['best_fit'] = 0
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('best_fit')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = np.mean([arr[i - total_x,o_col],arr[i - total_x,c_col]])
        y2 = np.mean([arr[i ,o_col],arr[i ,c_col]])
        
        temp_lst = get_best_fit_vals(y1,y2,total_x)
        arr[i,new_col] = compare_vals(lst = temp_lst,open_list = list(arr[i - total_x:i,o_col]),close_list = list(arr[i - total_x:i,c_col]),total_x = total_x)
            
    
    return pd.DataFrame(arr,columns = df.columns)   

def print_example(df,ind_list,ind = 30,delta = 240,trendline_lookback = 20):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind,' delta:',delta)
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta*2),:]['time'],
                        open=df.iloc[i - delta:i + (delta*2),:]['o'],
                        high=df.iloc[i - delta:i + (delta*2),:]['h'],
                        low=df.iloc[i - delta:i + (delta*2),:]['l'],
                        close=df.iloc[i - delta:i + (delta*2),:]['c']))


    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    
    fig.add_shape(type='line',
                    x0=df.iloc[i - trendline_lookback,:]['time'],
                    y0=df.iloc[i - trendline_lookback,:]['o'],
                    x1=df.iloc[i,:]['time'],
                    y1=df.iloc[i ,:]['o'],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
    )
    
    
    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
def get_max_min_open_or_close(df):
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    df['max_o_c'] = 0
    df['min_o_c'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')    
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,max_col] = max(arr[i,o_col],arr[i,c_col])
        arr[i,min_col] = min(arr[i,o_col],arr[i,c_col])
    return pd.DataFrame(arr,columns = df.columns)

def channel_indicator(df,lookback = 20):
    """Take the max of a bunch of consecutive candles and find the slope of the line
    Take the min of a bunch of consecutive candles and find the slope of the line
    
    """
    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,check_list,total_x,check_type = 'max'):

        for i in range(total_x):
            if check_type == 'max':
                if lst[i] >= check_list[i]:
                    pass
                else:
                    return 0
            if check_type == 'min':
                if lst[i] <= check_list[i]:
                    pass
                else:
                    return 0
        return 1
    
    
    print('GETTING CHANNEL INDICATOR')
    df['channel_indicator'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')
    new_col = df.columns.get_loc('channel_indicator')
    df['spread_indicator'] = 0
    spread_col = df.columns.get_loc('spread_indicator')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = arr[i - total_x,max_col]
        y2 = arr[i ,max_col]
        
        temp_lst1 = get_best_fit_vals(y1,y2,total_x)
        check_1 = compare_vals(lst = temp_lst1,
                                      check_list = list(arr[i - total_x:i,min_col]),
                                      total_x = total_x,
                                      check_type = 'max'
                                     )
        
        
        y1 = arr[i - total_x,min_col]
        y2 = arr[i ,min_col]
        
        temp_lst2 = get_best_fit_vals(y1,y2,total_x)
        check_2 = compare_vals(lst = temp_lst2,
                                      check_list = list(arr[i - total_x:i,max_col]),
                                      total_x = total_x,
                                      check_type = 'min'
                                     )   
        spread = np.array(temp_lst1) - np.array(temp_lst2)     
        arr[i,spread_col] = max(spread)
        
        
        if check_1 == 1 and check_2 == 1:
            arr[i,new_col] = 1
    
    return pd.DataFrame(arr,columns = df.columns)  


def get_support(df,lookup_range = 60,stop_range = 20000,lookup_range2 = 200):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup'] = 0
    df['support_indicator'] = 0
    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            if arr[i,c_col] == min(arr[i - lookup_range : i + lookup_range,c_col]):
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0:
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate


def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if arr[j,h_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate

def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if trade_id == 0:
                        if arr[j,l_col] < val - sl:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            trade_id = 1

                            break
                        if arr[j,h_col] >= val + tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            trade_id = 1
                            break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if arr[j,h_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses
def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)

file = 'XAU_USD_M1_2019-01-01_2022-01-31.csv'
#file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
#file = 'XAU_USD_S5/2020-08-01_2020-09-01.csv'
#file = 'GBP_JPY_S5/2020-08-01_2020-09-01.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'

def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    print(df.shape)
    return df

pair = 'GBP_JPY'
pair = 'EUR_USD'

granularity = 'M5'
df = load_df(pair = pair,granularity = granularity,start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))

granularity2 = 'M1'
df2 = load_df(pair = pair,granularity = granularity2,start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df2.head()



In [ ]:
temp = df[['o','h','l','c']].head().values
temp

In [ ]:
np.argmin(temp[:,0])

In [ ]:
def support_creation(df,lookup_range = 100,lookup_range2 = 800,lookup_range3 = 12,drop_thresh = .0005,bounce_thresh = .002,bounce_num = 2):
    df['support_check'] = 0
    col = df.columns.get_loc('support_check')
    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    arr = df.values
    trade_list = []
    for i in range(lookup_range,arr.shape[0]):
        bounce_list = []
        if arr[i,c_col] == min(arr[i - lookup_range:i+lookup_range3,c_col]):
            bounce_list.append([i])
            lower_range = arr[i,l_col]
            upper_range = arr[i,o_col]

            bounces = 1
            try:
                for j in range(i+lookup_range3,i + lookup_range2):
                    if i not in trade_list:
                        if arr[j,l_col] < lower_range - drop_thresh:
                            bounces = 100
                            break
                        if arr[j,l_col] >= lower_range - drop_thresh and arr[i,l_col] <= upper_range and bounces == bounce_num:
                            bounces = bounces + 1 
                            arr[j,col] = 1
                            trade_list.append(i)
                            break
                        #max(arr[bounce_list[-1]:j,h_col]) - arr[j,c_col] >= bounce_thresh
                        elif arr[j,l_col] >= lower_range - drop_thresh and arr[i,l_col] <= upper_range and min(arr[j-lookup_range3:j+ lookup_range3,l_col]) == arr[j,l_col] and bounces < bounce_num:
                            bounces = bounces + 1
                            bounce_list.append([j])
            except:
                pass

    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_check'] == 1].shape)
    return df

def support_creation(df,lookup_range = 100,lookup_range2 = 800,lookup_range3 = 12,drop_thresh = .0005,bounce_thresh = .002,bounce_num = 2):
    df['support_check'] = 0
    col = df.columns.get_loc('support_check')
    df['support_ind'] = 0
    col2 = df.columns.get_loc('support_ind')    
    df['support_ind2'] = 0
    col3 = df.columns.get_loc('support_ind2')    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    arr = df.values
    trade_list = []
    for i in range(lookup_range,arr.shape[0]):
        bounce_list = []
        if arr[i,l_col] == min(arr[i - lookup_range:i+lookup_range3,l_col]):
            bounce_list.append([i])
            lower_range = arr[i,l_col]
            upper_range = lower_range + arr[i,lw_col]

            bounces = 1
            try:
                
                for j in range(1,50):
                    min_ = min(arr[i +lookup_range3 + ((j-1) * 12) :i +lookup_range3 + ((j) * 12),l_col ]) 
                    arg_min = np.argmin(arr[i +lookup_range3 + ((j-1) * 12) :i +lookup_range3 + ((j) * 12),l_col ])
                    global_min = min(arr[i:i +lookup_range3 + ((j) * 12),l_col])
                    if min_ >= lower_range - drop_thresh and min_ <= upper_range and global_min >= lower_range - drop_thresh \
                    and max(arr[bounce_list[-1]:i +lookup_range3 + ((j-1) * 12) + arg_min,h_col]) - min_ >= bounce_thresh:
                        bounce_list.append()
                        bounces = bounces + 1
                        if bounces == bounce_num:
                            for k in range(i +lookup_range3 + (j * 12) , (i +lookup_range3 + (j * 12)) + lookup_range2):
                                if arr[k,l_col] >= lower_range - drop_thresh and arr[k,l_col] <= upper_range and i not in trade_list:
                                    arr[k,col] = 1
                                    arr[k,col2] = arr[i,c_col]
                                    arr[k,col3] = i
                                    trade_list.append(i)
                        
                        
                    
                    
            except:
                pass

    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_check'] == 1].shape)
    return df


def viz_wick_zoom_(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):

    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):
        i = ind_list[ind]   




        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   




        fig.add_vrect(x0=df['time'].iloc[df['support_ind2'].iloc[i] - 1], x1=df['time'].iloc[df['support_ind2'].iloc[i] + 1], 
                      annotation_text="SUPPORT",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="ENTRY",
                      fillcolor="green", opacity=0.25, line_width=0)


        

        fig.add_shape(type='line',
                    x0=df.iloc[i-delta,:]['time'],
                    y0=df['support_ind'].iloc[i],
                    x1=df.iloc[i+delta,:]['time'],
                    y1=df['support_ind'].iloc[i],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
        )           


        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta)
        i += 1
        

df = support_creation(df,lookup_range = 100,lookup_range2 = 800,lookup_range3 = 12,drop_thresh = .0005,bounce_thresh = .002,bounce_num = 3)

_func = 'support_check'
ind_list = list(df[df[_func] == 1].index)    

sl = 0.0005
tp = 0.002
start = 50
end = 55
delta = 100
viz_wick_zoom_(df,ind_list,start,end,delta,sl = sl,tp = tp)
df.head()



In [ ]:
_func = 'support_check'
ind_list = list(df[df[_func] == 1].index)    

sl = 0.0005
tp = 0.002
start = 40
end = 60
delta = 100
viz_wick_zoom_(df,ind_list,start,end,delta,sl = sl,tp = tp)
df.head()

In [ ]:
def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1:
            arr[i,new_col] = 1
    return pd.DataFrame(arr,columns = df.columns)

df2 = create_spike_trigger(df2,col = 'delta',lookback_threshold = 3600)
df = create_spike_trigger(df,col = 'delta',lookback_threshold = 1600)
df2.head()

In [ ]:
def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1:
            arr[i,new_col] = 1
    return pd.DataFrame(arr,columns = df.columns)

df2 = create_spike_trigger(df2,col = 'lower_wick',lookback_threshold = 60 * 24)
df2.head()

In [ ]:
df = create_spike_trigger(df,col = 'delta',lookback_threshold = 60 * 24)
df = create_spike_trigger(df,col = 'lower_wick',lookback_threshold = 60 * 24)

In [ ]:
20 * 24 * 5

In [ ]:
def test_viz(df,i,val,result,trade,sl,tp,delta = 60,buy = True):
    print('TP:',tp,'SL:',sl,'RR:',round(tp/sl,2))
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))   



    if buy:
        label = 'BUY'
    else:
        label = 'SELL'
        
    if result ==1:
        label = label + '_WIN'
        
    if result ==0:
        label = label + '_LOSS'
        
        
    fig.add_vrect(x0=df['time'].iloc[i-1], x1=df['time'].iloc[i], 
                  annotation_text=label,
                  fillcolor="green", opacity=0.25, line_width=0)

    

    fig.add_shape(type='line',
                x0=df.iloc[i-10,:]['time'],
                y0=val,
                x1=df.iloc[i+10,:]['time'],
                y1=val,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )    

    if buy:
        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val + tp,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val + tp,
                    line=dict(color='Green',),
                    xref='x',
                    yref='y'
        )            

        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val - sl,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val - sl,
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
        )   
    else:
        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val - tp,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val - tp,
                    line=dict(color='Green',),
                    xref='x',
                    yref='y'
        )            

        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val + sl,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val + sl,
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
        )   


    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
        
temp_df = final_df
ind_list = list(temp_df['entries'].iloc[0])
sl = temp_df['sl'].iloc[0]
entry_points = list(temp_df['entry_points'].iloc[0])
results = list(temp_df['results'].iloc[0])
trade_list = list(temp_df['trade_list'].iloc[0])
tp =  temp_df['tp'].iloc[0] 
i = 0
test_viz(df,i = ind_list[i],val = entry_points[i],result = results[i],trade = trade_list[i],sl = sl,tp = tp,delta = 20)
  

In [ ]:
def support_check(df,lookup_range = 100):
    df['support_check'] = 0
    col = df.columns.get_loc('support_check')
    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    
    arr = df.values
    
    for i in range(lookup_range,arr.shape[0]):
        if arr[i,c_col] == min(arr[i - lookup_range:i+lookup_range,c_col]):
            arr[i,col] = 1

    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_check'] == 1].shape)
    return df

def viz_wick_zoom_(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):

    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):
        i = ind_list[ind]   




        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="SUPPORT",
                      fillcolor="green", opacity=0.25, line_width=0)


        

        


        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta)
        i += 1
        

df = support_check(df,lookup_range = 100)

_func = 'support_check'
ind_list = list(df[df[_func] == 1].index)    

sl = 0.0005
tp = 0.002
start = 100
end = 130
delta = 100
viz_wick_zoom_(df,ind_list,start,end,delta,sl = sl,tp = tp)
df.head()

In [ ]:
(24* 60) / 5

In [ ]:
def strategy_tester_simple_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0008):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i+1,o_col] 

           # val = arr[i,l_col] + pip_delta
            trade_id = 0
            for j in range(i+1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses

def strategy_tester_simple_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0008):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col] 

           # val = arr[i,l_col] + pip_delta
            trade_id = 0
            for j in range(i + 1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses
def sig_candle(df,delta,delta2,delta3,buy = True):
    df['sig_candle'] = 0
    col = df.columns.get_loc('sig_candle')
    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    
    arr = df.values
    
    for i in range(arr.shape[0]):

        if buy and arr[i,delta_col] <= delta and arr[i,lw_col] >= delta2 and arr[i,c_col] < arr[i,o_col] and arr[i,o_col] >= arr[i - 500,o_col] + delta3 and arr[i,c_col] < arr[i,o_col] and arr[i,delta_col] >= delta:

            arr[i,col] = 1
        elif not buy and arr[i,delta_col] >= delta and arr[i,uw_col] >= delta2 and arr[i,c_col] > arr[i,o_col] and arr[i,o_col] >= arr[i - 500,o_col] + delta3:
            arr[i,col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['sig_candle'] == 1].shape)
    return df


def sig_candle(df,delta,delta2,delta3,buy = True):
    df['sig_candle'] = 0
    col = df.columns.get_loc('sig_candle')
    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    
    arr = df.values
    
    for i in range(arr.shape[0]):

        if buy and arr[i - 1,delta_col] >= delta and arr[i,c_col] + (delta/2) <= arr[i,o_col] and arr[i,o_col] >= arr[i - 300,o_col] + delta3:

            arr[i,col] = 1
        elif not buy and arr[i,delta_col] >= delta and arr[i,uw_col] >= delta2 and arr[i,c_col] > arr[i,o_col] and arr[i,o_col] >= arr[i - 500,o_col] + delta3:
            arr[i,col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['sig_candle'] == 1].shape)
    return df

def sig_candle(df,delta,delta2,delta3,buy = True):
    df['sig_candle'] = 0
    col = df.columns.get_loc('sig_candle')
    
    delta_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    
    arr = df.values
    
    for i in range(arr.shape[0]):

        if buy and arr[i,o_col] >= arr[i - 300,o_col] + delta3 and arr[i,c_col] <= arr[i - 6,o_col] - .002 and sum(arr[i - 100:i,col]) == 0 and arr[i,c_col] < arr[i,o_col] and arr[i,delta_col] >= delta:

            arr[i,col] = 1
        elif not buy and arr[i,delta_col] >= delta and arr[i,uw_col] >= delta2 and arr[i,c_col] > arr[i,o_col] and arr[i,o_col] >= arr[i - 500,o_col] + delta3:
            arr[i,col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['sig_candle'] == 1].shape)
    return df


delta = .0005
delta2 = .0005
delta3 = .005
df = sig_candle(df,delta,delta2,delta3,buy = True)

col = 'sig_candle'


results = []
cols = ['buy_or_sell','ending_val','sl','tp','win_rate','wins','losses']
final_df = pd.DataFrame([0])

for sl in sls2:
    for tp in tps2:
        #ending_val,win_rate,wins,losses = strategy_tester_buy_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        ending_val,win_rate,wins,losses = strategy_tester_simple_buy(df,col = col,sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False,pip_delta = pip_delta)
        results.append(['buy',ending_val,sl,tp,win_rate,wins,losses])
       # ending_val,win_rate,wins,losses,entries,entry_points,results_ = strategy_tester_sell_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        #results.append(['sell',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_])
        ending_val,win_rate,wins,losses = strategy_tester_simple_sell(df,col = col,sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False,pip_delta = pip_delta)
        results.append(['sell',ending_val,sl,tp,win_rate,wins,losses])        
        
final_df = pd.DataFrame(results,columns = cols)       
final_df = final_df.sort_values(by = 'ending_val',ascending = False)
final_df



In [ ]:
df.iloc[df[(df['delta'] >= delta) & (df['upper_wick'] <= delta2) & (df['c'] > df['o'])].index,:]

In [ ]:
temp_df

# TRIPPLE WICK

In [ ]:
def strategy_tester_buy_2(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i ,i + 10000):
                    below = 0
                    if arr[j,l_col] <= val:
                        below = 1
                        entries.append(j)
                    try:
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl and below == 1:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp and below == 1:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                    except:
                        pass
            except:
                pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list

def strategy_tester_sell_2(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i ,i + 10000):
                    below = 0
                    if arr[j,l_col] <= val:
                        below = 1
                        entries.append(j)
                    try:
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,h_col] >= val + sl and below == 1:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,l_col] <= val - tp and below == 1:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                    except:
                        pass
            except:
                pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list
def strategy_tester_buy_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
   
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    entry_check = []
    entry_points = []
    results = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
        
            val = arr[i,o_col] - (pip_delta * 2)
            entry_points.append(arr[i,o_col] - (pip_delta * 2))
            entries.append(i)
            trade_id = 0
            below = 0
            for j in range(i ,i + 10000):
                
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1
                                results.append(0)

                                break
                            elif arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                results.append(1)
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list



def strategy_tester_sell_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    entry_check = []
    entry_points = []
    results = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            val = arr[i,o_col] - pip_delta
            trade_id = 0
            below = 0
            for j in range(i + 1,i + 10000):
                
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1
                                results.append(0)

                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                results.append(1)
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list





def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0008):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col] 
            val = arr[i,o_col] - pip_delta
           # val = arr[i,l_col] + pip_delta
            trade_id = 0
            for j in range(i,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0008):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col] 
            val = arr[i,o_col] - pip_delta
           # val = arr[i,l_col] + pip_delta
            trade_id = 0
            for j in range(i + 1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses
def create_spike_tripple_wick(df,col = 'delta_max',pip_delta = .0005):
    print('THIS FUNCTION DOES NOT WORK...')
    df[col + '_tripple_wick'] = 0
    new_col = df.columns.get_loc(col + '_tripple_wick')
    l_col = df.columns.get_loc('l')
    o_col = df.columns.get_loc('o')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(20,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i-1,col_loc] >= pip_delta and arr[i,col_loc] >= pip_delta and arr[i + 1,l_col] <= arr[i,l_col]:# and arr[i,o_col] - arr[i,l_col] >= (pip_delta + (pip_delta * .25)) :#arr[i- 2,col_loc] >= pip_delta#arr[i,col_loc] >= pip_delta:# and arr[i -2,col_loc] >= pip_delta:# and arr[i + 1,l_col] <= arr[i,l_col]:
            arr[i,new_col] = 1

    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_tripple_wick'] == 1]
    print('SHAPE:',temp.shape)
    return df


def create_spike_tripple_wick_(df,col = 'delta_max',pip_delta = .0005):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_tripple_wick'] = 0
    new_col = df.columns.get_loc(col + '_tripple_wick')
    l_col = df.columns.get_loc('l')
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(20,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if  arr[i,col_loc] >= pip_delta and arr[i + 1,h_col] >= arr[i,l_col] + pip_delta and arr[i + 1,l_col] <= arr[i,l_col] + pip_delta:# and arr[i,o_col] - arr[i,l_col] >= (pip_delta + (pip_delta * .25)) :#arr[i- 2,col_loc] >= pip_delta#arr[i,col_loc] >= pip_delta:# and arr[i -2,col_loc] >= pip_delta:# and arr[i + 1,l_col] <= arr[i,l_col]:
            arr[i,new_col] = 1

    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_tripple_wick'] == 1]
    print('SHAPE:',temp.shape)
    return df

def create_spike_tripple_wick(df,col = 'delta_max',pip_delta = .0005):
    print('THIS FUNCTION DOES NOT WORK...')
    df[col + '_tripple_wick'] = 0
    new_col = df.columns.get_loc(col + '_tripple_wick')
    df[col + '_tripple_wick_entry'] = 0
    new_col2 = df.columns.get_loc(col + '_tripple_wick_entry')
    l_col = df.columns.get_loc('l')
    o_col = df.columns.get_loc('o')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(20,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i-1,col_loc] >= pip_delta and arr[i,o_col] - arr[i,l_col] >= (pip_delta * 2): #and arr[i,col_loc] >= pip_delta: and arr[i + 1,l_col] <= arr[i,l_col]:# and arr[i,o_col] - arr[i,l_col] >= (pip_delta + (pip_delta * .25)) :#arr[i- 2,col_loc] >= pip_delta#arr[i,col_loc] >= pip_delta:# and arr[i -2,col_loc] >= pip_delta:# and arr[i + 1,l_col] <= arr[i,l_col]:
            arr[i,new_col] = 1
            arr[i,new_col2] = arr[i,o_col] - (pip_delta * 2)

    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_tripple_wick'] == 1]
    print('SHAPE:',temp.shape)
    return df






pip_delta = .0008
df = create_spike_tripple_wick(df,col = 'lower_wick',pip_delta = pip_delta)
#df3 = df[df['o'] > df['c']]
col = 'lower_wick_tripple_wick'
df3 = df2.merge(df[['utc_timestamp',col,col + '_entry']],how = 'left',on = 'utc_timestamp')
print(df2.shape,df3.shape)

results = []
cols = ['buy_or_sell','ending_val','sl','tp','win_rate','wins','losses','entries','entry_points','results','trade_list']
final_df = pd.DataFrame([0])
for sl in sls2:
    for tp in tps2:
        #ending_val,win_rate,wins,losses = strategy_tester_buy_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        ending_val,win_rate,wins,losses,entries,entry_points,results_,trade_list = strategy_tester_buy_2(df3,col = col,sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        results.append(['buy',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_,trade_list])
       # ending_val,win_rate,wins,losses,entries,entry_points,results_ = strategy_tester_sell_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        #results.append(['sell',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_])
        ending_val,win_rate,wins,losses,entries,entry_points,results_,trade_list = strategy_tester_sell_2(df3,col = col,sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        results.append(['sell',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_,trade_list])        
        
final_df = pd.DataFrame(results,columns = cols)       
final_df = final_df.sort_values(by = 'ending_val',ascending = False)
final_df


        

In [ ]:
pip_delta

In [ ]:
def viz_wick_zoom_(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):

    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0015):
        i = ind_list[ind]   
        x = df[_func].iloc[i]



        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="I:" + str(i) + " trigger point:" + str(df['l'].iloc[i]) + " support at: " + str(df.iloc[x,:]['l']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[i-delta,:]['time'],
                        y0=df.iloc[x,:]['l'] + .0005,
                        x1=df.iloc[i + delta,:]['time'],
                        y1=df.iloc[x ,:]['l'] + .0005,
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        

        
        if buy:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['l'] + tp + pip_delta,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['l'] + tp + pip_delta,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['l'] +  pip_delta,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['l'] +  pip_delta,
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )   
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['l'] + pip_delta - sl,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['l'] + pip_delta - sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
            

        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta,sl = sl,tp = tp,_func = _func,buy = buy,pip_delta = pip_delta)
        i += 1
        
_func = 'lower_wick_tripple_wick'
ind_list = list(df[df[_func] == 1].index)    

sl = 0.0005
tp = 0.002
start = 30
end = 40
delta = 60
viz_wick_zoom_(df,ind_list,start,end,delta,sl = sl,tp = tp,_func =_func,buy = True,pip_delta = pip_delta)

In [ ]:
def viz_wick_zoom(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True):

    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True):
        i = ind_list[ind]   
        x = df[_func].iloc[i]



        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="I:" + str(i) + " trigger point:" + str(df['l'].iloc[i]) + " support at: " + str(df.iloc[x,:]['l']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[i-delta,:]['time'],
                        y0=df.iloc[x,:]['l'],
                        x1=df.iloc[i + delta,:]['time'],
                        y1=df.iloc[x ,:]['l'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        

        
        if buy:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['l'] + tp,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['l'] + tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['l'] - sl,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['l'] - sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
            
        else:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['h'] - tp,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['h'] - tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['h'] + sl,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['h'] + sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
                        
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta,sl = sl,tp = tp,_func = _func,buy = buy)
        i += 1
        
_func = 'lower_wick_tripple_wick'
ind_list = list(df[df[_func] == 1].index)    
sl = 0.0005 
tp = 0.0015
start = 30
end = 35
delta = 60
viz_wick_zoom(df,ind_list,start,end,delta,sl = sl,tp = tp,_func =_func,buy = True)

In [ ]:
df.head()

In [ ]:
def create_spike_trigger_retracement(df,col = 'delta_max',lookback_threshold = 3600,pip_delta = .0005):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger_retracement'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger_retracement')
    l_col = df.columns.get_loc('l')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1 and arr[i+1,l_col] <= arr[i,l_col] - pip_delta :
            arr[i,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_spike_trigger_retracement'] == 1]
    print('SHAPE:',temp.shape)
    return df

df = create_spike_trigger_retracement(df,col = 'lower_wick',lookback_threshold = int((2 * 24) ),pip_delta = .001)
df.head()

In [ ]:
(10 * 24 * 5) 

In [ ]:
12 * 5

# THIS WORKS DO NOT TOUCH

In [ ]:
def strategy_tester_buy_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col] - pip_delta
            trade_id = 0
            for j in range(i + 1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses


def strategy_tester_sell_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col] - pip_delta
            trade_id = 0
            for j in range(i + 1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses

def create_spike_trigger_retracement(df,col = 'delta_max',lookback_threshold = 3600,pip_delta = .0005):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger_retracement'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger_retracement')
    l_col = df.columns.get_loc('l')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1 and arr[i+1,l_col] <= arr[i,l_col] - pip_delta :
            arr[i,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_spike_trigger_retracement'] == 1]
    print('SHAPE:',temp.shape)
    return df


pip_delta = .0008
df = create_spike_trigger_retracement(df,col = 'lower_wick',lookback_threshold = int((2 * 24) ),pip_delta = pip_delta)

for sl in sls2:
    for tp in tps2:
        ending_val,win_rate,wins,losses = strategy_tester_buy_(df,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)



In [ ]:
df2.head()

In [ ]:
df2.head()

In [ ]:
df.head()

In [ ]:
col = 'lower_wick_spike_trigger_retracement'
df3 = df2.merge(df[['utc_timestamp',col]],how = 'left',on = 'utc_timestamp')
print(df2.shape,df3.shape)

In [ ]:
df3.head()

In [ ]:
df3[df3[col] == 1]

In [ ]:
df[df[col] == 1].shape

In [ ]:
df3.head()

# THIS WORKS DO NOT TOUCH

In [ ]:
def strategy_tester_buy_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            for j in range(i + 1,i + 10000):
                below = 0
                if arr[j,l_col] <= val:
                    below = 1
                    entries.append(j)
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl and below == 1:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp and below == 1:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses

def strategy_tester_buy_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    entry_check = []
    entry_points = []
    results = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            below = 0
            for j in range(i + 1,i + 10000):
                
                if arr[j,l_col] <= val:
                    below = 1
                    if i not in entry_check:
                        entries.append(j)
                        entry_check.append(i)
                        entry_points.append(val)
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl and below == 1:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1
                                results.append(0)

                                break
                            if arr[j,h_col] >= val + tp and below == 1:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                results.append(1)
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list
def strategy_tester_sell_(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,pip_delta = .0005):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(col + '_entry')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    entries = []
    entry_check = []
    entry_points = []
    results = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            below = 0
            for j in range(i + 1,i + 10000):
                
                if arr[j,l_col] <= val:
                    below = 1
                    if i not in entry_check:
                        entries.append(j)
                        entry_check.append(i)
                        entry_points.append(val)
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,h_col] >= val + sl and below == 1:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1
                                results.append(0)

                                break
                            if arr[j,l_col] <= val - tp and below == 1:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                results.append(1)
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses,entries,entry_points,results,trade_list

def create_spike_trigger_retracement(df,col = 'delta_max',lookback_threshold = 3600,pip_delta = .0005):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger_retracement'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger_retracement')
    df[col + '_spike_trigger_retracement_entry'] = 0
    new_col2 = df.columns.get_loc(col + '_spike_trigger_retracement_entry')
    l_col = df.columns.get_loc('l')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1 and arr[i+1,l_col] <= arr[i,l_col] - pip_delta :
            arr[i,new_col] = 1
            arr[i,new_col2] = arr[i,l_col] - pip_delta
            
    df = pd.DataFrame(arr,columns = df.columns)
    temp = df[df[col + '_spike_trigger_retracement'] == 1]
    print('SHAPE:',temp.shape)
    return df


pip_delta = .0008
df = create_spike_trigger_retracement(df,col = 'lower_wick',lookback_threshold = int((2 * 24) ),pip_delta = pip_delta)
#df3 = df[df['o'] > df['c']]
col = 'lower_wick_spike_trigger_retracement'
df3 = df2.merge(df[['utc_timestamp',col,col + '_entry']],how = 'left',on = 'utc_timestamp')
print(df2.shape,df3.shape)
results = []
cols = ['buy_or_sell','ending_val','sl','tp','win_rate','wins','losses','entries','entry_points','results','trade_list']
final_df = pd.DataFrame([0])
for sl in sls2:
    for tp in tps2:
        #ending_val,win_rate,wins,losses = strategy_tester_buy_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        ending_val,win_rate,wins,losses,entries,entry_points,results_,trade_list = strategy_tester_buy_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        results.append(['buy',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_,trade_list])
       # ending_val,win_rate,wins,losses,entries,entry_points,results_ = strategy_tester_sell_(df3,col = 'lower_wick_spike_trigger_retracement',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'l',pr = False,pip_delta = pip_delta)
        #results.append(['sell',ending_val,sl,tp,win_rate,wins,losses,entries,entry_points,results_])
        
final_df = pd.DataFrame(results,columns = cols)       
final_df = final_df.sort_values(by = 'ending_val',ascending = False)
final_df


In [ ]:
final_df[final_df['tp'] < .005]

In [ ]:
final_df

In [ ]:
len(final_df['wins'].iloc[0])

In [ ]:
final_df['wins'].iloc[0]

In [ ]:
df3.head()

In [ ]:
def test_viz(df,i,val,result,trade,sl,tp,delta = 60,buy = True):
    print('TP:',tp,'SL:',sl,'RR:',round(tp/sl,2))
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))   



    if buy:
        label = 'BUY'
    else:
        label = 'SELL'
        
    if result ==1:
        label = label + '_WIN'
        
    if result ==0:
        label = label + '_LOSS'
        
        
    fig.add_vrect(x0=df['time'].iloc[i], x1=df['time'].iloc[i+1], 
                  annotation_text=label,
                  fillcolor="green", opacity=0.25, line_width=0)

    fig.add_vrect(x0=df['time'].iloc[trade], x1=df['time'].iloc[trade+1], 
                  annotation_text='TRIGGER ' + str(df['lower_wick'].iloc[trade]),
                  fillcolor="blue", opacity=0.25, line_width=0)
    

    fig.add_shape(type='line',
                x0=df.iloc[i-10,:]['time'],
                y0=val,
                x1=df.iloc[i+10,:]['time'],
                y1=val,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )    

    if buy:
        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val + tp,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val + tp,
                    line=dict(color='Green',),
                    xref='x',
                    yref='y'
        )            

        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val - sl,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val - sl,
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
        )   
    else:
        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val - tp,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val - tp,
                    line=dict(color='Green',),
                    xref='x',
                    yref='y'
        )            

        fig.add_shape(type='line',
                    x0=df.iloc[i-10,:]['time'],
                    y0=val + sl,
                    x1=df.iloc[i+10,:]['time'],
                    y1=val + sl,
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
        )   


    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
        
temp_df = final_df
ind_list = temp_df['entries'].iloc[0]
sl = temp_df['sl'].iloc[0]
entry_points = list(temp_df['entry_points'].iloc[0])
results = list(temp_df['results'].iloc[0])
trade_list = list(temp_df['trade_list'].iloc[0])
tp =  temp_df['tp'].iloc[0] 
i = 5
test_viz(df3,i = ind_list[i],val = entry_points[i],result = results[i],trade = trade_list[i],sl = sl,tp = tp,delta = 50)


In [ ]:
temp_df

In [ ]:
temp_df = final_df[final_df['tp'] < .004]
temp_df.head()

In [ ]:
ind_list = temp_df['wins'].iloc[0]
ind_list[:15]

In [ ]:
ind_list = temp_df['entries'].iloc[0]
ind_list[:15]

In [ ]:
def viz_wick_zoom(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0008):

    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True,pip_delta = .0008):
        
        
        
        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   




        label = 'IDK'

        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text=label,
                      fillcolor="green", opacity=0.25, line_width=0)


        

        
        if buy:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['o'] + tp,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['o'] + tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-10,:]['time'],
                        y0=df.iloc[i,:]['o'] - sl,
                        x1=df.iloc[i+10,:]['time'],
                        y1=df.iloc[i ,:]['o'] - sl,
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )   
            
 
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta,sl = sl,tp = tp,_func = _func,buy = buy,pip_delta = pip_delta)
        i += 1
        
_func = 'lower_wick_spike_trigger_retracement'
ind_list = list(df3[df3[_func] == 1].index)   
ind_list = final_df['entries'].iloc[0]
sl = 0.0005 
sl = final_df['sl'].iloc[0]
tp = 0.001
tp =  final_df['tp'].iloc[0]
start = 0
end = 10
delta = 60
viz_wick_zoom(df3,ind_list,start,end,delta,sl = sl,tp = tp,_func =_func,buy = True)

In [ ]:
df3[df3[_func] == 1].shape

In [ ]:
df[df['lower_wick_spike_trigger'] == 1].shape

In [ ]:
df.head()

In [ ]:
60 * 24

In [ ]:
60 * 24

In [ ]:
df3.shape

# THIS WORKS DO NOT TOUCH

In [ ]:
def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    new_col = df.columns.get_loc(col + '_spike_trigger')
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,new_col]) == 0 and arr[i,new_col] != 1:
            arr[i,new_col] = 1
    return pd.DataFrame(arr,columns = df.columns)

def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp,' COL',col)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            for j in range(i + 1,i + 10000):
                try:
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
                except:
                    pass

    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size,' ACTUAL NUM TRADES:',len(trade_list), 'ACTUAL WINS:',len(wins))
    return np.median(vals),win_rate,wins,losses


df = create_spike_trigger(df,col = 'delta',lookback_threshold = int((20 * 24) ))
df = create_spike_trigger(df,col = 'lower_wick',lookback_threshold = int((20 * 24) ))
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]

sls2 = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps2 = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]

df3 = df[(df['o'] > df['c'])]

col = 'lower_wick_spike_trigger'
col = 'delta_spike_trigger'
col = 'lower_wick_spike_trigger'
for sl in sls2:
    for tp in tps2:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df3,col = col,sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False)



In [ ]:
df2[df2['delta_spike_trigger'] == 1].shape

In [ ]:
new_col = df.columns.get_loc(col + '_spike_trigger')

In [ ]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
    pr = True):
    
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buya(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate

def strategy_tester_sella(df,df2,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    s = time.time()
    lookup_ind = df.columns.get_loc('support_lookupa')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    time_col = df.columns.get_loc('utc_timestamp')
    
    arr = df.values
    arr2 = df2.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    

    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            ind = arr[i,lookup_ind]
            val = arr[ind,l_col]
            #val = arr[i,c_col]
            stamp = arr[i,time_col]
            temp_arr = arr2[arr2[:,time_col] >= stamp,:][:10000,:]
            entry_ind = 0
            for j in range(temp_arr.shape[0]):
                
                if temp_arr[j,l_col] <= val:
                    entry_ind = 1
                if i not in trade_list:
                    if temp_arr[j,h_col] >= val + sl and entry_ind == 1:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if temp_arr[j,l_col] <= val - tp and entry_ind == 1:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
                        
                        
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    e = time.time()
    print('SIM TIME:',round((e-s)/60,4),'MINUTES')
    return np.median(vals),win_rate,wins,losses


def get_support2a(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookupa'] = 0
    df['support_indicatora'] = 0

    lookup_col = df.columns.get_loc('support_lookupa')
    new_col = df.columns.get_loc('support_indicatora')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0 and arr[j,trading_session_col] == 1:
                            for k in range(j + 1,j+30):
                                if arr[k,signal_col] == 1:
   
                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
                        elif lookup_check == 1:
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicatora'] == 1].shape)
    return df

def get_supporta(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookupa'] = 0
    df['support_indicatora'] = 0

    lookup_col = df.columns.get_loc('support_lookupa')
    new_col = df.columns.get_loc('support_indicatora')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:
                          
   
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                       
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicatora'] == 1].shape)
    return df

def get_supporta(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookupa'] = 0
    df['support_indicatora'] = 0

    lookup_col = df.columns.get_loc('support_lookupa')
    new_col = df.columns.get_loc('support_indicatora')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)

    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_

                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:# and arr[j,trading_session_col] == 1:
                            for k in range(j + 1,j+30):
                                if arr[k,signal_col] == 1:
   
                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
                        elif lookup_check == 1:
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicatora'] == 1].shape)
    return df

def get_supporta(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    df['support_lookupa'] = 0
    df['support_indicatora'] = 0

    lookup_col = df.columns.get_loc('support_lookupa')
    new_col = df.columns.get_loc('support_indicatora')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0 and arr[j,o_col] - val > .2 and arr[j,c_col] >= val - .5:# and arr[j,trading_session_col] == 1:
                       
   
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicatora'] == 1].shape)
    return df





cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0

else:
    pip_threshold = .003
    pip_threshold2 = .0015
    
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 450
lookup_range2 = 288 # 1 day
lookup_range2 = 288
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'

sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]

buy_or_sell = 1
df = get_supporta(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    
trade_strategy = 'support_indicatora'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_sella(df,df2,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)




In [ ]:
def strategy_tester_sella2(df,df2,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    s = time.time()
    lookup_ind = df.columns.get_loc('support_lookupa')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    time_col = df.columns.get_loc('utc_timestamp')
    
    arr = df.values

    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    

    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            ind = arr[i,lookup_ind]
            val = arr[ind,l_col]

            for j in range(i,i + 1000):

                if i not in trade_list:
                    if arr[j,h_col] >= val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
                        
                        
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    e = time.time()
    print('SIM TIME:',round((e-s)/60,4),'MINUTES')
    return np.median(vals),win_rate,wins,losses

def strategy_tester_buya2(df,df2,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    s = time.time()
    lookup_ind = df.columns.get_loc('support_lookupa')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    time_col = df.columns.get_loc('utc_timestamp')
    
    arr = df.values

    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    

    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            ind = arr[i,lookup_ind]
            val = arr[ind,l_col]
          #  val = arr[i,c_col]

            for j in range(i,i + 1000):

                if i not in trade_list:
                    if arr[j,l_col] <= val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
                        
                        
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    e = time.time()
    print('SIM TIME:',round((e-s)/60,4),'MINUTES')
    return np.median(vals),win_rate,wins,losses

def get_supporta(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    df['support_lookupa'] = 0
    df['support_indicatora'] = 0

    lookup_col = df.columns.get_loc('support_lookupa')
    new_col = df.columns.get_loc('support_indicatora')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:# and arr[j,o_col] - val - pip_threshold2 > .1:# and arr[j,trading_session_col] == 1:== 1:
                       
   
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicatora'] == 1].shape)
    return df





cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0

else:
    pip_threshold = .003
    pip_threshold2 = .0015
    
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 450
lookup_range2 = 288 # 1 day
lookup_range2 = 288
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'

sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]

buy_or_sell = 1
df = get_supporta(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    
trade_strategy = 'support_indicatora'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_sella2(df,df2,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        lst = [instrument,timeframe,0,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)
        ending_val,win_rate,wins,losses = strategy_tester_buya2(df,df2,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        lst = [instrument,timeframe,1,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)



In [ ]:
r = .25
val = 14000
for i in range(1,24):
    val = val + (val * r)
    print(i,val)

# POSSIBLY WINNING STRATEGY NEED MORE DD:
## SELL SUPPORT / BUY RESISTANCE

In [ ]:
df.head()

In [ ]:
288 * 5

In [ ]:
(60 * 24) / 5

In [ ]:
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
num_trades = 400
risk_size = .01
for sl in sls:
    for tp in tps:
        ending_val = strategy_tester_sella(df,col = 'support_indicatora',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)
        

In [ ]:
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
num_trades = 400
risk_size = .01
for sl in sls:
    for tp in tps:
        ending_val = strategy_tester_buya(df,col = 'support_indicatora',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)
        

# SIM STRATEGY 2
## Above Support Sell

In [ ]:
def get_supportb(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing',func_ = 'b'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup' + func_] = 0
    df['support_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('support_lookup' + func_)
    new_col = df.columns.get_loc('support_indicator' + func_)
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    signal2 = df.columns.get_loc('bullish_engulfing')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0 and arr[j,trading_session_col] == 1:
                            for k in range(j + 1,j+60):
                                if arr[k,signal_col] == 1:# and arr[k,signal2] == 1:
   
                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
                        elif lookup_check == 1:
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator' + func_] == 1].shape)
    return df

cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .25

else:
    pip_threshold = .003
    pip_threshold2 = .0015


    
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]

lookup_range = 288 # 1 day
lookup_range = 350
lookup_range2 = 288 # 1 day
lookup_range2 = 350
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bearish_engulfing'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bullish_tls'


df = get_supportb(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    
trade_strategy = 'support_indicatorb'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
sl = .05
tp = 1.1

ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)     
print('NUM WINS:',len(wins))
viz_support_zoom(df,ind_list = wins,start = 0,end = 29,delta = 100,sl = sl,tp = tp,_func = 'b',buy = False)


In [ ]:
def viz_support(df,start,end,_func = 'a'):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240,_func = 'a'):
        i = ind_list[ind]   
        x = df['support_lookup'+ _func].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator' + _func] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 400,_func = _func)
        i += 1
        
        
viz_support(df = df,start = 5,end = 30,_func = 'b')



In [ ]:
def print_support_example(df,ind_list,ind = 30,delta = 240,col = 'support_indicatorb'):
    i = ind_list[ind]   
    x = df[col].iloc[i]


    print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

    fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                        open=df.iloc[x - delta:i + delta,:]['o'],
                        high=df.iloc[x - delta:i + delta,:]['h'],
                        low=df.iloc[x - delta:i + delta,:]['l'],
                        close=df.iloc[x - delta:i + delta,:]['c']))   






    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)

    fig.add_shape(type='line',
                    x0=df.iloc[x,:]['time'],
                    y0=df.iloc[x,:]['c'],
                    x1=df.iloc[i,:]['time'],
                    y1=df.iloc[x ,:]['c'],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
    )

    fig.add_shape(type='line',
                    x0=df.iloc[i-12,:]['time'],
                    y0=df.iloc[i,:]['c'],
                    x1=df.iloc[i+12,:]['time'],
                    y1=df.iloc[i ,:]['c'],
                    line=dict(color='Blue',),
                    xref='x',
                    yref='y'
    )

    fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
    fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
    fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
        
        
col = 'support_indicatorb'
temp_df = df[df[col] == 1]
print('SHAPE:',temp_df.shape)
ind_list = temp_df.index

i =15

print_support_example(df,ind_list,ind = i,delta = 400,col = col)
 


In [ ]:
ind_list[15]
df[col].iloc[ind_list[15]]

In [ ]:
1000 / 5

In [ ]:
1000 / 60

In [ ]:
(400 * 5) / 60

In [ ]:
df.head()

In [ ]:
ending_val = strategy_tester_sella(df,col = 'support_indicatorb',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)



In [ ]:

cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]

# 0 = sell, 1 = buy
buy_or_sell = 0
trade_strategy = 'support_indicatorb'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate = strategy_tester_sella(df,col = 'support_indicatorb',sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)



In [ ]:
(12 * 60)/5

In [ ]:
df.head()

In [ ]:
trigger_signals = ['bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle']



# SIM Strategy 3 THIS WORKS!!! This is actually breaking support then coming back up to retest before going down
## Large Candle near support

In [ ]:
def get_supportc(df,
                 lookup_range = 1500,
                 lookup_range2 = 2400,
                 stop_range = 2200,
                 pip_threshold = .005,
                 pip_threshold2 = .005,
                 trigger_signal =None,
                 candle_size = 0,
                 wick_size = 0,
                 bullish = 0,
                 func_ = 'c'):
    if trigger_signal not in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle'):
        raise Exception("needs to be in list:",('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle')) 
    print('GETTING SUPPORT INDICATOR')
    print('PARAMS:','lookup range',lookup_range,' lookup range2',lookup_range2,' stop range',stop_range,' pip thresh',
          pip_threshold, ' pip thresh 2',pip_threshold2,' trigger signal',trigger_signal,' candle size',candle_size,' wick_size',wick_size,' bullish',bullish)
    s = time.time()
    o_col = df.columns.get_loc('o') 
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    bearish_engulfing_col = df.columns.get_loc('bearish_engulfing')
    bullish_engulfing_col = df.columns.get_loc('bullish_engulfing')
    bearish_tls_col = df.columns.get_loc('bearish_tls')
    bullish_tls_col = df.columns.get_loc('bullish_tls')
    if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls'):
        triger_col = df.columns.get_loc(trigger_signal)
    else:
        triger_col = df.columns.get_loc('bearish_engulfing')
    
    df['support_lookup' + func_] = 0
    df['support_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('support_lookup' + func_)
    new_col = df.columns.get_loc('support_indicator' + func_)

    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        if arr[j,c_col] < val - pip_threshold2:
                            lookup_check = 1
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        
                        if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0 and arr[j,h_col] >= val: #has to be near support
                            actual_candle_size = abs(arr[j,c_col] - arr[j,o_col])
                            actual_wick_size = abs(arr[j,h_col] - arr[j,l_col])
                            if arr[j,c_col] - arr[j,o_col] >= 0:
                                bull = 1
                            else:
                                bull = 0
                                
                            ## TRIGGER FILTER
                            if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls') and arr[j,trigger_col] == 1 and actual_candle_size >= candle_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                            elif trigger_signal == 'large_candle' and actual_candle_size >= candle_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                                
                            elif trigger_signal == 'large_wick' and actual_wick_size >= wick_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break        
                                
                           # if (arr[j,c_col] - arr[j,o_col] >= .25)
      


        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator' + func_] == 1].shape)
    return df

def get_resistancec(df,
                 lookup_range = 1500,
                 lookup_range2 = 2400,
                 stop_range = 2200,
                 pip_threshold = .005,
                 pip_threshold2 = .005,
                 trigger_signal =None,
                 candle_size = 0,
                 wick_size = 0,
                 bullish = 0,
                 func_ = 'c'):
    if trigger_signal not in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle'):
        raise Exception("needs to be in list:",('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle')) 
    print('GETTING RESISTANCE INDICATOR')
    print('PARAMS:','lookup range',lookup_range,' lookup range2',lookup_range2,' stop range',stop_range,' pip thresh',
          pip_threshold, ' pip thresh 2',pip_threshold2,' trigger signal',trigger_signal,' candle size',candle_size,' wick_size',wick_size,' bullish',bullish)
    s = time.time()
    o_col = df.columns.get_loc('o') 
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    bearish_engulfing_col = df.columns.get_loc('bearish_engulfing')
    bullish_engulfing_col = df.columns.get_loc('bullish_engulfing')
    bearish_tls_col = df.columns.get_loc('bearish_tls')
    bullish_tls_col = df.columns.get_loc('bullish_tls')
    if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls'):
        triger_col = df.columns.get_loc(trigger_signal)
    else:
        triger_col = df.columns.get_loc('bearish_engulfing')
    
    df['resistance_lookup' + func_] = 0
    df['resistance_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('resistance_lookup' + func_)
    new_col = df.columns.get_loc('resistance_indicator' + func_)

    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            max_ = max(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                min_ = min(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,c_col] > val + pip_threshold2:
                            lookup_check = 1          
                            
                        if arr[j,l_col] >= val - pip_threshold2 and lookup_check == 0 and arr[j,l_col] <= val: #has to be near support
                            actual_candle_size = abs(arr[j,c_col] - arr[j,o_col])
                            actual_wick_size = abs(arr[j,h_col] - arr[j,l_col])
                            if arr[j,c_col] - arr[j,o_col] >= 0:
                                bull = 1
                            else:
                                bull = 0
                                
                            ## TRIGGER FILTER
                            if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls') and arr[j,trigger_col] == 1 and actual_candle_size >= candle_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                            elif trigger_signal == 'large_candle' and actual_candle_size >= candle_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                                
                            elif trigger_signal == 'large_wick' and actual_wick_size >= wick_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break        
                                
                           # if (arr[j,c_col] - arr[j,o_col] >= .25)
      


        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator' + func_] == 1].shape)
    return df

def get_supportc(df,
                 lookup_range = 1500,
                 lookup_range2 = 2400,
                 stop_range = 2200,
                 pip_threshold = .005,
                 pip_threshold2 = .005,
                 trigger_signal =None,
                 candle_size = 0,
                 wick_size = 0,
                 bullish = 0,
                 func_ = 'c'):
    if trigger_signal not in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle'):
        raise Exception("needs to be in list:",('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle')) 
    print('GETTING SUPPORT INDICATOR')
    print('PARAMS:','lookup range',lookup_range,' lookup range2',lookup_range2,' stop range',stop_range,' pip thresh',
          pip_threshold, ' pip thresh 2',pip_threshold2,' trigger signal',trigger_signal,' candle size',candle_size,' wick_size',wick_size,' bullish',bullish)
    s = time.time()
    o_col = df.columns.get_loc('o') 
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    bearish_engulfing_col = df.columns.get_loc('bearish_engulfing')
    bullish_engulfing_col = df.columns.get_loc('bullish_engulfing')
    bearish_tls_col = df.columns.get_loc('bearish_tls')
    bullish_tls_col = df.columns.get_loc('bullish_tls')
    if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls'):
        triger_col = df.columns.get_loc(trigger_signal)
    else:
        triger_col = df.columns.get_loc('bearish_engulfing')
    
    df['support_lookup' + func_] = 0
    df['support_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('support_lookup' + func_)
    new_col = df.columns.get_loc('support_indicator' + func_)

    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):

                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        
                        if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0 and arr[j,h_col] >= val: #has to be near support
                            actual_candle_size = abs(arr[j,c_col] - arr[j,o_col])
                            actual_wick_size = abs(arr[j,h_col] - arr[j,l_col])
                            if arr[j,c_col] - arr[j,o_col] >= 0:
                                bull = 1
                            else:
                                bull = 0
                                
                            ## TRIGGER FILTER
                            if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls') and arr[j,trigger_col] == 1 and actual_candle_size >= candle_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                            elif trigger_signal == 'large_candle' and actual_candle_size >= candle_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                                
                            elif trigger_signal == 'large_wick' and actual_wick_size >= wick_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break        
                                
                           # if (arr[j,c_col] - arr[j,o_col] >= .25)
      
                      #  if arr[j,c_col] < val - pip_threshold2:
                       #     lookup_check = 1

        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator' + func_] == 1].shape)
    return df

def get_resistancec(df,
                 lookup_range = 1500,
                 lookup_range2 = 2400,
                 stop_range = 2200,
                 pip_threshold = .005,
                 pip_threshold2 = .005,
                 trigger_signal =None,
                 candle_size = 0,
                 wick_size = 0,
                 bullish = 0,
                 func_ = 'c'):
    if trigger_signal not in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle'):
        raise Exception("needs to be in list:",('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle')) 
    print('GETTING RESISTANCE INDICATOR')
    print('PARAMS:','lookup range',lookup_range,' lookup range2',lookup_range2,' stop range',stop_range,' pip thresh',
          pip_threshold, ' pip thresh 2',pip_threshold2,' trigger signal',trigger_signal,' candle size',candle_size,' wick_size',wick_size,' bullish',bullish)
    s = time.time()
    o_col = df.columns.get_loc('o') 
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    
    bearish_engulfing_col = df.columns.get_loc('bearish_engulfing')
    bullish_engulfing_col = df.columns.get_loc('bullish_engulfing')
    bearish_tls_col = df.columns.get_loc('bearish_tls')
    bullish_tls_col = df.columns.get_loc('bullish_tls')
    if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls'):
        triger_col = df.columns.get_loc(trigger_signal)
    else:
        triger_col = df.columns.get_loc('bearish_engulfing')
    
    df['resistance_lookup' + func_] = 0
    df['resistance_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('resistance_lookup' + func_)
    new_col = df.columns.get_loc('resistance_indicator' + func_)

    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            max_ = max(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == max_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                min_ = min(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
       
                            
                        
                        if arr[j,h_col] >= val - pip_threshold2 and lookup_check == 0 and arr[j,l_col] <= val: #has to be near support
                            actual_candle_size = abs(arr[j,c_col] - arr[j,o_col])
                            actual_wick_size = abs(arr[j,h_col] - arr[j,l_col])
                            if arr[j,c_col] - arr[j,o_col] >= 0:
                                bull = 1
                            else:
                                bull = 0
                                
                            ## TRIGGER FILTER
                            if trigger_signal in ('bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls') and arr[j,trigger_col] == 1 and actual_candle_size >= candle_size:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                            elif trigger_signal == 'large_candle' and actual_candle_size >= candle_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break    
                                
                            elif trigger_signal == 'large_wick' and actual_wick_size >= wick_size and bull == bullish:
                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break              
                                
                           # if (arr[j,c_col] - arr[j,o_col] >= .25)
      
                        #if arr[j,c_col] > val + pip_threshold2:
                         #   lookup_check = 1   

        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator' + func_] == 1].shape)
    return df


cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .5
    pip_threshold = .5
    pip_threshold2 = .25

else:
    pip_threshold = .003
    pip_threshold2 = .0015
    
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]

lookup_range = 288 # 1 day
lookup_range = 350
lookup_range2 = 288 # 1 day
lookup_range2 = 350
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bearish_engulfing'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bullish_tls'

trigger_signals = ['bearish_engulfing','bullish_engulfing','bearish_tls','bullish_tls','large_wick','large_candle']

candle_sizes = [0,.10,.15,.20,.25]
wick_sizes = [0,.5,.10,.15,.20]
bullish_ = [0,1]






trigger_signal = 'large_candle'
trigger_signal = 'large_wick'
trigger_signal = 'large_candle'
trigger_signal = 'large_wick'
trigger_signal = 'large_candle'
candle_size = .25

wick_size = .35
bullish = 1



df = get_supportc(df,
                  lookup_range = lookup_range,
                  lookup_range2 = lookup_range2,
                  stop_range = stop_range,
                  pip_threshold = pip_threshold,
                  pip_threshold2 = pip_threshold2,
                 trigger_signal = trigger_signal,
                 candle_size = candle_size,
                 wick_size = wick_size,
                 bullish = bullish,                  
                  func_ = 'c')
df = get_resistancec(df,
                  lookup_range = lookup_range,
                  lookup_range2 = lookup_range2,
                  stop_range = stop_range,
                  pip_threshold = pip_threshold,
                  pip_threshold2 = pip_threshold2,
                 trigger_signal = trigger_signal,
                 candle_size = candle_size,
                 wick_size = wick_size,
                 bullish = bullish,                  
                  func_ = 'c')
    

cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]

# 0 = sell, 1 = buy
buy_or_sell = 0
trade_strategy = 'support_indicatorc'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
len(wins)

In [ ]:
df[df['resistance_indicatorc'] == 1]

In [ ]:
buy_or_sell = 0
#trade_strategy = 'support_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)

In [ ]:
viz_support_zoom(df = df,start = 20,end = 30,delta = 50,sl = sl,tp = tp,_func = 'c',buy = False )
#viz_support(df = df,start = 5,end = 30,_func = 'c')

In [ ]:
num_trades = 250
for sl in sls:
    for tp in tps:
        ending_val,win_rate = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)

# SIM Strategy 4 THIS doesnt work!!!
## Large Candle Breaking through support then take buys
## ADDING: and(arr[j,c_col] - arr[j,l_col] <= .01) i.e low less than 1 pip away from close increases probability of winning.

In [ ]:
def get_supportd(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,lookback_range = 144,func_ = 'd'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    o_col = df.columns.get_loc('o') 
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup' + func_] = 0
    df['support_indicator' + func_] = 0

    lookup_col = df.columns.get_loc('support_lookup' + func_)
    new_col = df.columns.get_loc('support_indicator' + func_)

    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        
                        if arr[j,o_col] > val and arr[j,c_col] < val and lookup_check == 0 and (arr[j,o_col] - arr[j,c_col] >= .2) and(arr[j,c_col] - arr[j,l_col] <= .01): #has to be near support
      
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                        #elif arr[j,c_col] < val:
                         #   lookup_check = 1

        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator' + func_] == 1].shape)
    return df

cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .7
    pip_threshold2 = .3

else:
    pip_threshold = .003
    pip_threshold2 = .0015
    
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]

lookup_range = 288 # 1 day
lookup_range = 350
lookup_range2 = 288 # 1 day
lookup_range2 = 350
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bearish_engulfing'
signal = 'bullish_tls'
signal = 'bullish_engulfing'
signal = 'bullish_tls'


df = get_supportd(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,lookback_range = 144,func_ = 'd')
    

cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]

# 0 = sell, 1 = buy
buy_or_sell = 0
trade_strategy = 'support_indicatord'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
print('actual_number_of_trades',actual_number_of_trades)
starting_val = 10000
instrument = 'GBP_JPY'
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        #ending_val,win_rate = strategy_tester_sella(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        ending_val,win_rate = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        lst = [instrument,timeframe,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)



In [ ]:
def viz_support_zoom(df,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True):
        i = ind_list[ind]   
        x = df['support_lookup'+ _func].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="I:" + str(i) + " trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[i-delta,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i + delta,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )
        
        if buy:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] + tp,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] + tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] - sl,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] - sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
            
        else:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] - tp,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] - tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] + sl,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] + sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
                        
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator' + _func] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta,sl = .05,tp = .15,_func = _func,buy = buy)
        i += 1
        
        
#viz_support_zoom(df = df,start = 20,end = 30,delta = 50,sl = .05,tp = .25,_func = 'd',buy = True )


In [ ]:
def viz_support_zoom(df,ind_list,start,end,delta,sl = .05,tp = .15,_func = 'd',buy = True):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240,sl = .05,tp = .15,_func = 'd',buy = True):
        i = ind_list[ind]   
        x = df['support_lookup'+ _func].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[i - delta:i + delta,:]['o'],
                            high=df.iloc[i - delta:i + delta,:]['h'],
                            low=df.iloc[i - delta:i + delta,:]['l'],
                            close=df.iloc[i - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="I:" + str(i) + " trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[i-delta,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i + delta,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )
        
        if buy:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] + tp,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] + tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] - sl,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] - sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
            
        else:
            
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] - tp,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] - tp,
                        line=dict(color='Green',),
                        xref='x',
                        yref='y'
        )            
        
            fig.add_shape(type='line',
                        x0=df.iloc[i-5,:]['time'],
                        y0=df.iloc[i,:]['c'] + sl,
                        x1=df.iloc[i+5,:]['time'],
                        y1=df.iloc[i ,:]['c'] + sl,
                        line=dict(color='Black',),
                        xref='x',
                        yref='y'
        )   
                        
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[i - delta:i + delta,:]['time'],y=df.iloc[i - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            

    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = delta,sl = sl,tp = tp,_func = _func,buy = buy)
        i += 1
        
sl = .1
tp = 1.3

ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)     
print('NUM WINS:',len(wins))
viz_support_zoom(df,ind_list = wins,start = 0,end = 29,delta = 100,sl = sl,tp = tp,_func = 'c',buy = False)


In [ ]:
len(wins)

In [ ]:
viz_support_zoom(df = df,start = 20,end = 30,delta = 50,sl = .05,tp = .25,_func = 'd',buy = True )
viz_support(df = df,start = 5,end = 30,_func = 'd')

In [ ]:
viz_support(df = df,start = 5,end = 30,_func = 'd')

In [ ]:
viz_support(df = df,start = 0,end = 10,_func = 'c')

In [ ]:
temp_lst.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])

In [ ]:
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
num_trades = 400
risk_size = .01
for sl in sls:
    for tp in tps:
        ending_val = strategy_tester_sella(df,col = 'support_indicatorb',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)
        
        

In [ ]:
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
num_trades = 400
risk_size = .01
for sl in sls:
    for tp in tps:
        ending_val = strategy_tester_sella(df,col = 'support_indicatorb',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)
        
        

In [ ]:
sls = [.05,.075,.1,.125,.15,.2,.25]
tps = [.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
num_trades = 400
risk_size = .01
for sl in sls:
    for tp in tps:
        ending_val = strategy_tester_sella(df,col = 'support_indicatorb',sl = sl,tp = tp,num_trades = num_trades,risk_size = .01,entry = 'c',pr = False)
        

In [ ]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print('RUNNING BUY SIMULATOR','SL:',sl,'TP:',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)
    print()

    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals)

In [ ]:
120 / 5

# SIM

In [ ]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy6(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)
    print()

    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals)


def get_support6(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6'] = 0
    df['support_indicator6'] = 0
    df['entry_point6'] = 0
    lookup_col = df.columns.get_loc('support_lookup6')
    new_col = df.columns.get_loc('support_indicator6')
    entry_col = df.columns.get_loc('entry_point6')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:
      
                                
                              #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                            
                            arr[j,entry_col] = val - pip_threshold2
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .003
    pip_threshold2 = .0015


In [ ]:
final_df = pd.read_csv('SIM_' + file)
final_df.head()

In [ ]:
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)

In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]

lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
final_df = pd.DataFrame([0])



cols = [
    
   'ending_val'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'sl'
    ,'tp'
    ,'num_trades'
    ,'risk_size' 
    
]


final_df = pd.read_csv('SIM_' + file)
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,' pip threshold:',pip_threshold,' pip threshold2:',pip_threshold2,' lookup range:',lookup_range,' lookup range2:',lookup_range2,' stop range:',stop_range)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    temp_lst = []
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            temp_lst.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])

                    temp_df = pd.DataFrame(temp_lst,columns = cols)
                    if final_df.shape[0] == 1:
                        final_df = temp_df
                    else:
                        final_df = final_df.append(temp_df)
                    final_df.to_csv('SIM_' + file,index = False)
                            
                            
final_list          




In [ ]:
final_df = pd.DataFrame([0])
final_df.shape[0]

In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005]
pip_thresholds = [.003]

lookup_ranges = [1500]
lookup_range2s = [1000]
stop_ranges = [10000]

sls = [.0005,.00075,.001]
tps = [.0015,.002,.0025]
num_trades = 200
risk_size = .01
final_df = pd.DataFrame([0])
cols = [
    
   'ending_val'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'sl'
    ,'tp'
    ,'num_trades'
    ,'risk_size' 
    
]
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,' pip threshold:',pip_threshold,' pip threshold2:',pip_threshold2,' lookup range:',lookup_range,' lookup range2:',lookup_range2,' stop range:',stop_range)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    temp_lst = []
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            temp_lst.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                    temp_df = pd.DataFrame(temp_lst,columns = cols)
                    if final_df.shape[0] == 1:
                        final_df = temp_df
                    else:
                        final_df = final_df.append(temp_df)
                    final_df.to_csv('SIM_' + file,index = False)
                            
                            
final_list          


In [ ]:
ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)

In [ ]:
1500 /60

In [ ]:
def viz_support(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support(df = df,start = 50,end = 70)



In [ ]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,entry = 'c'):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
    print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades)
    vals = []


    acc_val = 10000

    risk_size = .02

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
        vals.append(final_val)
    print()
    print('AVG SIM:',sum(vals) / len(vals))    
    print('MEDIAN SIM:',np.median(vals))
    
    return trade_res,trade_list
def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300):
    print('RUNNING SELL SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
    #num_trades = len(trade_list)
    print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades)
    vals = []


    acc_val = 10000

    risk_size = .02

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
        vals.append(final_val)
    print()
    print('AVG SIM:',sum(vals) / len(vals))  
    print('MEDIAN SIM:',np.median(vals))
    
    return trade_res,trade_list



#trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator',sl = .001,tp = .003)    
#trade_res,trade_list = strategy_tester_sell(df,col = 'support_indicator',sl = .001,tp = .003) 



# IDEA 2
## Support is Low of last 24 hours that moves up at least 50 PIPs and back down within a window of 12 - 36 hours

In [ ]:


def get_support2(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .002 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup2'] = 0
    df['support_indicator2'] = 0
    lookup_col = df.columns.get_loc('support_lookup2')
    new_col = df.columns.get_loc('support_indicator2')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if min(arr[i + lookup_range2:j - 120,l_col]) >= val:
                        if min(arr[j - 60:j,l_col]) <= val - pip_threshold and arr[j,h_col] >= val + pip_threshold2 and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator2'] == 1].shape)
    return df
cur = 'NON JPY'
cur = 'JPY'
cur = 'NON JPY'
print(cur)

if cur == 'JPY':
    pip_threshold = .5
else:
    pip_threshold = .003
    pip_threshold2 = .001
    
df = get_support2(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 5000,pip_threshold = pip_threshold ,pip_threshold2 = pip_threshold2 )

 


In [ ]:
trade_res,trade_list = strategy_tester(df,col = 'support_indicator2',sl = .001,tp = .001) 



In [ ]:
def viz_support2(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup2'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        #fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_7500'],line=dict(color='Orange',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator2'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support2(df = df,start = 0,end = 15)


In [ ]:
2200/60

# IDEA 3 
## Same as idea 2 but needs to drop a certain amount below support to trigger a buy

In [ ]:


def get_support3(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,drop_threshold = .0015,drop_timeframe = 100 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup3'] = 0
    df['support_indicator3'] = 0
    lookup_col = df.columns.get_loc('support_lookup3')
    new_col = df.columns.get_loc('support_indicator3')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                        if min(arr[j:j + drop_timeframe,l_col]) <= val - drop_threshold:
                        
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator3'] == 1].shape)
    return df
cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .5
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support3(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 4000,pip_threshold = pip_threshold,drop_threshold = drop_threshold,drop_timeframe = 60  )
df.head()


In [ ]:
def viz_support3(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup3'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator3'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support3(df = df,start = 40,end = 70)

# Idea 4 sell support?

In [ ]:


def get_support4(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,drop_threshold = .0015,drop_timeframe = 100 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup4'] = 0
    df['support_indicator4'] = 0
    lookup_col = df.columns.get_loc('support_lookup4')
    new_col = df.columns.get_loc('support_indicator4')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator4'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support4(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 4000,pip_threshold = pip_threshold,drop_threshold = drop_threshold,drop_timeframe = 60  )
df.head()


In [ ]:
def viz_support4(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup4'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator4'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support4(df = df,start = 100,end = 130)



In [ ]:
df.head()

# Idea 5 double bottom within 24 hours

In [ ]:


def get_support5(df,lookup_range = 1500,stop_range = 2200,pip_threshold = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup5'] = 0
    df['support_indicator5'] = 0
    lookup_col = df.columns.get_loc('support_lookup5')
    new_col = df.columns.get_loc('support_indicator5')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)

    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + 30,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + 30,i + stop_range):
                    #print('VAL',(max(arr[i : j,c_col]) - min_))
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator5'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .3
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .0005
    
df = get_support5(df,lookup_range = 1500,stop_range = 1500,pip_threshold = pip_threshold)
df.head()


In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator5',sl = .001,tp = .003) 

In [ ]:
def viz_support5(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup5'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator5'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support5(df = df,start = 10,end = 40)



In [ ]:
2400 / 60

# Idea 6 within ~20 pips 2 - 5 days later

In [ ]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy6(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)
    print()

    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals)


def get_support6(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6'] = 0
    df['support_indicator6'] = 0
    df['entry_point6'] = 0
    lookup_col = df.columns.get_loc('support_lookup6')
    new_col = df.columns.get_loc('support_indicator6')
    entry_col = df.columns.get_loc('entry_point6')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:
      
                                
                              #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                            
                            arr[j,entry_col] = val - pip_threshold2
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .003
    pip_threshold2 = .0015

    
    
#df = get_support6(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 10000,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
#df.head()




In [ ]:
final_list = []
ending_val = 100000

pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]

sls[.0005,.00075,.001,.00125,.0015,.002,.0025]
tps[.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    for sl in sls:
                        for tp in tps:
                            


ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6',pr = False)
final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])



In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'entry_point6',pr = False)
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                            
final_list                            


In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]

lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                            
final_list                            


In [ ]:
ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6',pr = False)
val

In [ ]:
trade_res,trade_list = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6')


In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'c') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'c') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 


In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 

In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0007,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0007,tp = .004,num_trades = 400,entry = 'l')

In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 

In [ ]:
np.median([1,2,3])

In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .004,num_trades = 400) 


In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .002,num_trades = 400) 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .002,num_trades = 400) 



In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .003,num_trades = 400) 

In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .001,num_trades = 400) 

# TRADE THAT WORKS
## EUR/USD 5 pips below support entry, support is 2-5 days after first reversal
## 10 pip sl and 20 pip tp

# Another trade that works
##     pip_threshold = .003
##    pip_threshold2 = .0015
## EUR/USD support is 2-5 days after first reversal
## 5 pip sl and 40 pip tp

# Idea 6a
## moves up 5 pips from bottom

In [ ]:


def get_support6a(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6a'] = 0
    df['support_indicator6a'] = 0
    lookup_col = df.columns.get_loc('support_lookup6a')
    new_col = df.columns.get_loc('support_indicator6a')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    mn_lst = []
    for i in range(arr.shape[0]):

        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
  
                        if arr[j,l_col] <= val and lookup_check == 0: #/- pip_threshold2:
                            for k in range(j,j + 120):
                                local_min = min(arr[j:k + 1,l_col])
                                if arr[k,c_col] >= local_min + pip_threshold2:


                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6a'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1

else:
    pip_threshold = .003
    pip_threshold2 = .0005

    
    
df = get_support6a(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 7500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
df.head()


In [ ]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6a',sl = .001,tp = .004,num_trades = 500) 


In [ ]:
def viz_support6a(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup6a'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator6a'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support6a(df = df,start = 0,end = 30)

In [ ]:
"trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c'])

In [ ]:
def viz_support6(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup6'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator6'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support6(df = df,start = 0,end = 30)


In [ ]:
sum(trade_res)

In [ ]:
len(trade_res)

In [ ]:
def strategy_tester(df,col = 'support_indicator6',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df['support_indicator6'] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,h_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester(df,col = 'support_indicator6',sl = .1,tp = 1)    


# Idea 7
## Goes beyond support then goes the same amount above support
## Sell once it hits the same amount above support

In [ ]:
def get_support7(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup7'] = 0
    df['support_indicator7'] = 0
    lookup_col = df.columns.get_loc('support_lookup7')
    new_col = df.columns.get_loc('support_indicator7')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if min(arr[i:j-60,l_col]) >= val:
                            if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:# and max(arr[j:j + 120,h_col])
                          #  if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0 and min(arr[i+ 30:j,l_col]) > val:


                                  #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator7'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .35
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support7(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 7500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)

In [ ]:
def viz_support7(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup7'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator7'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support7(df = df,start = 60,end = 90)

In [ ]:
def strategy_tester_buy(df,col = 'support_indicator7',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-1 * sl)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(tp)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator7',sl = .2,tp = .5)    


# Idea 8 double bottoms
## needs to go a certain amount below

In [ ]:


def get_support8(df,lookup_range = 1500,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005  ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup8'] = 0
    df['support_indicator8'] = 0
    lookup_col = df.columns.get_loc('support_lookup8')
    new_col = df.columns.get_loc('support_indicator8')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)

    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range,c_col])
            #max_ = min(arr[i : i + 30,c_col])
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range,i + lookup_range + stop_range):
                    #print('VAL',(max(arr[i : j,c_col]) - min_))
                    if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold :
                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator8'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .3
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support8(df,lookup_range = 1000,stop_range = 2500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
df.head()

In [ ]:
def viz_support8(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup8'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator8'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support8(df = df,start = 90,end = 120)

In [ ]:
def strategy_tester_buy(df,col = 'support_indicator8',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-1 * sl)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(tp)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator8',sl = .1,tp = .7)   

In [ ]:
1500 / 60

In [ ]:
700/60

In [ ]:
(5 / 2)

In [ ]:
from random import random

def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)
vals = []

num_trades = 500
acc_val = 10000
rr = 4
risk_size = .02
win_rate = .22
for i in range(30):
    
    final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
    vals.append(final_val)
print()
print('AVG SIM:',sum(vals) / len(vals))


